<a href="https://colab.research.google.com/github/mralamdari/CV-Yolo/blob/main/YOLO_v3_ObjectDetection_TensorFlow_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np

!wget -P model_data https://pjreddie.com/media/files/yolov3.weights

['--2022-08-13 04:50:47--  https://pjreddie.com/media/files/yolov3.weights',
 'Resolving pjreddie.com (pjreddie.com)... 128.208.4.108',
 'Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.',
 'HTTP request sent, awaiting response... 200 OK',
 'Length: 248007048 (237M) [application/octet-stream]',
 'Saving to: ‘model_data/yolov3.weights’',
 '',
 '',
 'yolov3.weights        0%[                    ]       0  --.-KB/s               ',
 'yolov3.weights        0%[                    ] 151.70K   705KB/s               ',
 'yolov3.weights        0%[                    ]   1.07M  2.48MB/s               ',
 'yolov3.weights        3%[                    ]   7.24M  11.2MB/s               ',
 'yolov3.weights        6%[>                   ]  15.82M  18.2MB/s               ',
 'yolov3.weights       10%[=>                  ]  24.52M  22.6MB/s               ',
 'yolov3.weights       14%[=>                  ]  33.41M  25.6MB/s               ',
 'yolov3.weights       17%[==>       

In [8]:
with open('model_data/yolov3.weights', 'rb') as wf:
    major, minor, revision, seen, _ = np.fromfile(wf, dtype=np.int32, count=5)

    conv_weights = np.fromfile(wf, dtype=np.float32, count=2)
